In [23]:
def get_stationinfo():
    from pathlib import Path
    import pandas as pd
    import xarray as xr
    
    data_dir = Path('../data')
    
    # def get_stationInfo():
    rsl = xr.load_dataset(data_dir / 'rsl_hawaii.nc')

    # Create a list to store rows as dictionaries
    data = []

    # Iterate over the data and store rows in the list
    for i in range(len(rsl.record_id)):
        data.append({'lat': rsl.lat[i].values,
                     'lon': rsl.lon[i].values,
                     'station_name': str(rsl.station_name[i].values),
                     'record_id': rsl.record_id[i].values})

        # Convert the list of dictionaries to a DataFrame
        station_info = pd.DataFrame(data)

        # add column for offsetting the text labels
        station_info['offsetlon'] = 0.2
        station_info['offsetlat'] = 0.2
        station_info['ha'] = 'left'

        # remove everything after the comma in station_name
        station_info['station_name'] = station_info['station_name'].str.split(',').str[0]

        # set Nawiliwli offsetlat to be 0.5
        station_info.loc[station_info['station_name']=='Nawiliwili', 'offsetlat'] = 0.4
        station_info.loc[station_info['station_name']=='Mokuoloe', 'offsetlat'] = 0.4
        station_info.loc[station_info['station_name']=='Hilo', 'offsetlat'] = 0.3
        station_info.loc[station_info['station_name']=='Hilo', 'offsetlon'] = 0.3
        station_info.loc[station_info['station_name']=='Kawaihae', 'ha'] = 'right'
        station_info.loc[station_info['station_name']=='Kawaihae', 'offsetlon'] = -0.5
        station_info.loc[station_info['station_name']=='Kawaihae', 'offsetlat'] = -0.5
        station_info.loc[station_info['station_name']=='Kaumalapau', 'ha'] = 'right'
        station_info.loc[station_info['station_name']=='Kaumalapau', 'offsetlon'] = 0
        station_info.loc[station_info['station_name']=='Kaumalapau', 'offsetlat'] = -0.6
        station_info.loc[station_info['station_name']=='Barbers Point', 'ha'] = 'right'
        station_info.loc[station_info['station_name']=='Barbers Point', 'offsetlon'] = -0.5
        station_info.loc[station_info['station_name']=='Barbers Point', 'offsetlat'] = -0.2
        station_info.loc[station_info['station_name']=='Honolulu', 'ha'] = 'right'
        station_info.loc[station_info['station_name']=='Honolulu', 'offsetlon'] = -0.1
        station_info.loc[station_info['station_name']=='Honolulu', 'offsetlat'] = -0.6

        station_info['fontcolor'] = 'gray'


    return station_info

In [24]:
station_info = get_stationinfo()
station_info

,lat,lon,station_name,record_id,offsetlon,offsetlat,ha,fontcolor
0,23.867,193.71,French Frigate,14,0.2,0.2,left,gray
1,28.217,182.633,Midway,50,0.2,0.2,left,gray
2,16.75,190.483,Johnston,52,0.2,0.2,left,gray
3,21.307,202.133,Honolulu,57,-0.1,-0.6,right,gray
4,21.967,200.65,Nawiliwili,58,0.2,0.4,left,gray
5,20.9,203.533,Kahului,59,0.2,0.2,left,gray
6,19.733,204.933,Hilo,60,0.3,0.3,left,gray
7,21.433,202.2,Mokuoloe,61,0.2,0.4,left,gray
8,21.322,201.88,Barbers Point,547,-0.5,-0.2,right,gray
9,20.783,203.0,Kaumalapau,548,0.0,-0.6,right,gray


In [ ]:
def add_zebra_frame(ax, lw=2, segment_length=0.5, crs=ccrs.PlateCarree()):
    # Get the current extent of the map
    left, right, bot, top = ax.get_extent(crs=crs)

    # Calculate the nearest 0 or 0.5 degree mark within the current extent
    left_start = left - left % segment_length
    bot_start = bot - bot % segment_length

    # Adjust the start if it does not align with the desired segment start
    if left % segment_length >= segment_length / 2:
        left_start += segment_length
    if bot % segment_length >= segment_length / 2:
        bot_start += segment_length

    # Extend the frame slightly beyond the map extent to ensure full coverage
    right_end = right + (segment_length - right % segment_length)
    top_end = top + (segment_length - top % segment_length)

    # Calculate the number of segments needed for each side
    num_segments_x = int(np.ceil((right_end - left_start) / segment_length))
    num_segments_y = int(np.ceil((top_end - bot_start) / segment_length))

    # Draw horizontal stripes at the top and bottom
    for i in range(num_segments_x):
        color = 'black' if (left_start + i * segment_length) % (2 * segment_length) == 0 else 'white'
        start_x = left_start + i * segment_length
        end_x = start_x + segment_length
        ax.hlines([bot, top], start_x, end_x, colors=color, linewidth=lw, transform=crs)

    # Draw vertical stripes on the left and right
    for j in range(num_segments_y):
        color = 'black' if (bot_start + j * segment_length) % (2 * segment_length) == 0 else 'white'
        start_y = bot_start + j * segment_length
        end_y = start_y + segment_length
        ax.vlines([left, right], start_y, end_y, colors=color, linewidth=lw, transform=crs)

In [ ]:
def plot_map(vmin, vmax, palette, xlims, ylims):
    """
    Plot a map of the magnitude of sea level change.

    Parameters:
    vmin (float): Minimum value for the color scale.
    vmax (float): Maximum value for the color scale.
    xlims (tuple): Tuple of min and max values for the x-axis limits.
    ylims (tuple): Tuple of min and max values for the y-axis limits.

    Returns:
    fig (matplotlib.figure.Figure): The matplotlib figure object.
    ax (matplotlib.axes._subplots.AxesSubplot): The matplotlib axes object.
    crs (cartopy.crs.Projection): The cartopy projection object.
    cmap (matplotlib.colors.Colormap): The colormap used for the plot.
    """
    crs = ccrs.PlateCarree()
    fig, ax = plt.subplots(figsize=(10, 5), subplot_kw={'projection': crs})
    ax.set_xlim(xlims)
    ax.set_ylim(ylims)

    cmap = palette

    ax.coastlines()
    ax.add_feature(cfeature.LAND, color='lightgrey')

    return fig, ax, crs

In [ ]:
def plot_map_grid(vmin, vmax, xlims, ylims, nrows, ncols):
    """
    Plot a map of the magnitude of sea level change.

    Parameters:
    vmin (float): Minimum value for the color scale.
    vmax (float): Maximum value for the color scale.
    xlims (tuple): Tuple of min and max values for the x-axis limits.
    ylims (tuple): Tuple of min and max values for the y-axis limits.

    Returns:
    fig (matplotlib.figure.Figure): The matplotlib figure object.
    ax (matplotlib.axes._subplots.AxesSubplot): The matplotlib axes object.
    crs (cartopy.crs.Projection): The cartopy projection object.
    cmap (matplotlib.colors.Colormap): The colormap used for the plot.
    """
    crs = ccrs.PlateCarree()
    fig, axs = plt.subplots(nrows, ncols, figsize=(10, 10), subplot_kw={'projection': crs})
    for ax in axs.flat:
        ax.set_xlim(xlims)
        ax.set_ylim(ylims)
    
        ax.coastlines()
        ax.add_feature(cfeature.LAND, color='lightgrey')

    return fig, axs, crs

In [ ]:
def plot_zebra_frame(ax, lw=5, segment_length=2, crs=ccrs.PlateCarree()):
    """
    Plot a zebra frame on the given axes.

    Parameters:
    - ax: The axes object on which to plot the zebra frame.
    - lw: The line width of the zebra frame. Default is 5.
    - segment_length: The length of each segment in the zebra frame. Default is 2.
    - crs: The coordinate reference system of the axes. Default is ccrs.PlateCarree().
    """
    # Call the function to add the zebra frame
    add_zebra_frame(ax=ax, lw=lw, segment_length=segment_length, crs=crs)
    # add map grid
    gl = ax.gridlines(draw_labels=True, linestyle=':', color='black',
                      alpha=0.5,xlocs=ax.get_xticks(),ylocs=ax.get_yticks())
    #remove labels from top and right axes
    gl.top_labels = False
    gl.right_labels = False

In [28]:
def plot_thin_map_hawaii(labelStations=False):
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature

    # Create the plot
    fig = plt.figure(figsize=(10, 7), constrained_layout=True)
    ax = plt.axes(projection=ccrs.PlateCarree())

    # Add features to the map
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.COASTLINE, linewidth=0.25)
    ax.add_feature(cfeature.BORDERS, linestyle=':')

    gl = ax.gridlines(draw_labels=True, linewidth=0.25, color='gray', alpha=0.5, linestyle='--')
    gl.xlabel_style = {'size': 8}
    gl.ylabel_style = {'size': 8}

    # Set the extent to focus on Hawaii and surrounding areas
    ax.set_extent([-179, -153, 15, 30])  # Adjust to focus on Hawaii

    station_info = get_stationinfo()
    station_label = {}

    if labelStations:
        # Add labels to the stations
        for i, row in station_info.iterrows():
            ax.scatter(row['lon'], row['lat'], color='black', s=10, transform=ccrs.PlateCarree())
            station_label[i] = ax.text(row['lon'] + row['offsetlon'], row['lat'] + row['offsetlat'], row['station_name'],
                    ha=row['ha'], va='center', transform=ccrs.PlateCarree(), fontsize=8, color=row['fontcolor'])

    return fig, ax, station_info, station_label
